<a href="https://colab.research.google.com/github/sagarpatiler/NLP/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 981 kB 60.9 MB/s 
     |████████████████████████████████| 263 kB 68.2 MB/s 
     |████████████████████████████████| 3.8 MB 62.7 MB/s 
     |████████████████████████████████| 1.3 MB 50.1 MB/s 
     |████████████████████████████████| 468 kB 64.0 MB/s 
     |████████████████████████████████| 7.6 MB 53.3 MB/s 
     |████████████████████████████████| 880 kB 56.0 MB/s 
     |████████████████████████████████| 163 kB 71.8 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=fabf2ffdd2bc680ac91088c75cc034bc1fa7328093a9c13c9fa926fd681271ac
  Stored in directory: /root/.cache/pip/wheels/9c/1c/1b/6df2db85720b8f5c6ea5e3ae37313cfc656f248abf910b7cfd
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=07ecf8c9476de2f2ef5d888

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import ktrain
from ktrain import text
tf.__version__

'2.9.2'

In [ ]:
df_train=pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx",dtype=str)
df_test=pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx",dtype=str)

In [ ]:
df_train.dtypes

Reviews      object
Sentiment    object
dtype: object

In [ ]:
df_train.head(5)

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=df_train, text_column='Reviews', label_columns='Sentiment',
                   val_df = df_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [ ]:
model=text.text_classifier(name='distilbert',train_data=train,preproc=preproc)

Is Multi-Label? False
maxlen is 400


Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

done.


In [ ]:
learner=ktrain.get_learner(train_data=train,val_data=val,model=model,batch_size=6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4167/4167 [==============================] - 1576s 375ms/step - loss: 0.2849 - accuracy: 0.8835 - val_loss: 0.1678 - val_accuracy: 0.9428
Epoch 2/2
4167/4167 [==============================] - 1568s 375ms/step - loss: 0.1594 - accuracy: 0.9416 - val_loss: 0.0665 - val_accuracy: 0.9813


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.save('/content/drive/My Drive/projects/DistillBert_ktrain')

In [ ]:
data = ['I like her']

In [ ]:
predictor.predict(data, return_proba=True)

1/1 [==============================] - 1s 1s/step


array([[0.05826981, 0.9417302 ]], dtype=float32)